In [1]:
using RRIFT, Perfusion, Statistics
using Plots, Interact

HTML{String}("<script>\n// Immediately-invoked-function-expression to avoid global variables.\n(function() {\n    var warning_div = document.getElementById(\"webio-warning-5550842406326858366\");\n    var hide = function () {\n        var script = document.getElementById(\"webio-setup-6266122873047729647\");\n        var parent = script && script.parentElement;\n        var grandparent = parent && parent.parentElement;\n        if (grandparent) {\n            grandparent.style.display = \"none\";\n        }\n        warning_div.style.display = \"none\";\n    };\n    if (typeof Jupyter !== \"undefined\") {\n        console.log(\"WebIO detected Jupyter notebook environment.\");\n        // Jupyter notebook.\n        var extensions = (\n            Jupyter\n            && Jupyter.notebook.config.data\n            && Jupyter.notebook.config.data.load_extensions\n        );\n        if (extensions && extensions[\"webio-jupyter-notebook\"]) {\n            // Extension already loaded.\n            console.log(\"Jupyter WebIO nbextension detected; not loading ad-hoc.\");\n            hide();\n            return;\n        }\n    } else if (window.location.pathname.includes(\"/lab\")) {\n        // Guessing JupyterLa\n        console.log(\"Jupyter Lab detected; make sure the @webio/jupyter-lab-provider labextension is installed.\");\n        hide();\n        return;\n    }\n})();\n\n</script>\n<p\n    id=\"webio-warning-5550842406326858366\"\n    class=\"output_text output_stderr\"\n    style=\"padding: 1em; font-weight: bold;\"\n>\n    Unable to load WebIO. Please make sure WebIO works for your Jupyter client.\n    For troubleshooting, please see <a href=\"https://juliagizmos.github.io/WebIO.jl/latest/providers/ijulia/\">\n    the WebIO/IJulia documentation</a>.\n    <!-- TODO: link to installation docs. -->\n</p>\n")

# Pre-processing

Before applying RRIFT, there's a couple of pre-processing steps including:
1. Load imaging data from the VFA DICOM files & compute T1 maps
1. Load imaging data form the DCE DICOM files & convert the DCE-MRI signal into tracer concentration
    + Note to mybinder users: This step requires a lot of memory, so sadly your journey will end here, but the output from the remaining cells can still be viewed.
1. Extract signal-time curves from the tumour, muscle, and artery for subsequent model fitting

In [2]:
chosen_study_uid = RRIFT.gbm_study_uids[8]

dicom_folders = download_invivo_studies(chosen_study_uid, destination = "./data/tcga-gbm-dicom")

# Extract the vfa and dce folders from `dicom_folders`
vfa_folder = dicom_folders.vfa_folders[1]
dce_folder = dicom_folders.dce_folders[1]

println("VFA: $vfa_folder")
println("DCE: $dce_folder")

VFA: ./data/tcga-gbm-dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.304604545029494418165835320551/vfa
DCE: ./data/tcga-gbm-dicom/1.3.6.1.4.1.14519.5.2.1.4591.4001.304604545029494418165835320551/dce


## T1 mapping

In [3]:
# Load VFA data
vfa = RRIFT.load_vfa_dicom(folder = vfa_folder)

println("""
Some information for VFA data:
    - It is a named tuple with keys: $(keys(vfa))
    - Number of flip angles: $(length(vfa.angles))
    - Value of flip angles, in degrees: $(sort(round.(rad2deg.(vfa.angles))))
    - Size of signal data: $(size(vfa.signal))
    - Repetition time, in ms: $(vfa.TR)
""")

Some information for VFA data:
    - It is a named tuple with keys: (:signal, :angles, :TR)
    - Number of flip angles: 6
    - Value of flip angles, in degrees: [2.0, 5.0, 10.0, 15.0, 20.0, 25.0]
    - Size of signal data: (256, 256, 16, 6)
    - Repetition time, in ms: 5.044



T1 mapping is provided by the `fit_relaxation` function from the `Perfusion` package. 
There are three algorithms for T1 mapping:
1. Non-linear least squares fitting of the spoiled gradient echo equation
1. Linear least squares fitting with [DESPOT1](https://onlinelibrary.wiley.com/doi/full/10.1002/mrm.20314)
1. Iterative fitting with [NOVIFAST](https://ieeexplore.ieee.org/document/8371285)

The fitting algorithm is selected by passing either `:nls`, `:despot`, or `:novifast` as the first argument.
This notebook uses DESPOT1 because that is what the paper used. I wasn't aware of NOVIFAST when I wrote the paper, or else I probably would've used it instead of DESPOT1.

**Note:** The `fit_relaxation` function expects the first argument to be a symbol followed by keyword arguments. 
The keywords are `signal`, `angles`, and `TR`. 
In others words, it could've been written as: `fit_relaxation(:despot, signal = vfa.signal, angles = vfa.angles, TR = vfa.TR)`.
However, since the keys in `vfa` match the function's keywords, we can just splat it with `...`. 

In [4]:
# Compute T1 maps using DESPOT1
relaxation_maps = fit_relaxation(:despot; vfa...).estimates

# The variable `relaxation_maps` contains the keys: T1 & M0
keys(relaxation_maps)

(:M0, :T1)

In [5]:
# Prepare the maps for plotting
is_dark = false # Set this to true if using a dark-background theme
plotbg = is_dark ? (background_color = RGB(0.067, 0.067, 0.067), ) : (background_color = RGB(1, 1, 1), )

M0 = crop(relaxation_maps.M0);
T1 = crop(relaxation_maps.T1);

In [ ]:
# Show the M0 and T1 maps
num_slices = size(relaxation_maps.T1)[3]
@manipulate for slice in 1:num_slices
    p1 = heatmap(M0[:,:,slice], c=:cinferno, yflip=true, aspect_ratio=:equal, clim=(0, 15000); title="M0", axis=nothing, plotbg...)
    p2 = heatmap(T1[:,:,slice], c=:cinferno, yflip=true, aspect_ratio=:equal, clim=(500, 2000); title="T1 (ms)", axis=nothing, plotbg...)
    plot(p1, p2, layout=(1,2))
end

## DCE-MRI signal to concentration conversion

The T1 map is used to convert the DCE-MRI signal into tracer concentration.
The `Perfusion` package provides the function `signal_to_concentration` for this purpose.

**!! Warning !!**  
These steps require about ~2 Gb of memory. This is (currently) more than what mybinder can handle, so the next steps will likely cause the kernel to restart. The output for the next cells is shown so running them is not necessary.

In [7]:
# Load the DCE-MRI data
dce = RRIFT.load_dce_dicom(folder = dce_folder)

println("""
Some information for DCE data:
    - It is a named tuple with keys: $(keys(dce))
    - Size of signal data: $(size(dce.signal))
    - Number of timepoints/frames: $(length(dce.timepoints))
    - Flip angle, in degrees: $(rad2deg(dce.angle))
    - Repetition time, in ms: $(dce.TR)
""")

Some information for DCE data:
    - It is a named tuple with keys: (:signal, :timepoints, :TR, :angle)
    - Size of signal data: (256, 256, 16, 70)
    - Number of timepoints/frames: 70
    - Flip angle, in degrees: 20.0
    - Repetition time, in ms: 5.044



In [8]:
r1 = 3.3/1000 # Relaxivity of Gd-DTPA at 3T, in mM/ms. Ref: PMID 16481903
BAF = 3 # Bolus arrival frame, i.e. the frame at which the tracer arrives in the imaging volume
R10 = 1 ./ relaxation_maps.T1 # The function wants R1 instead of T1. Who are we to argue?

concentration = signal_to_concentration(dce.signal; angle = dce.angle, TR = dce.TR, R10 = R10, BAF = BAF, r1 = r1)
size(concentration)

(256, 256, 16, 70)

## Wrapper function for T1 mapping and signal-concentration conversion

The functions in the previous section were shown for educational purposes. 
The `compute_concentration` function wraps the previous steps together:

In [9]:
computed = compute_concentration(vfa_folder = vfa_folder, dce_folder = dce_folder)

println("""
Some information for the computed maps:
    - It is a named tuple with keys: $keys(computed)
    - Does it have the same T1 map as we computed earlier? $(all(@. (computed.T1 == relaxation_maps.T1)[!isnan(computed.T1)]))
    - Does it have the same concentration as computed earlier? $(all(@. (computed.ct == concentration)[!isnan(concentration)]))
""")

Some information for the computed maps:
    - It is a named tuple with keys: keys(computed)
    - Does it have the same T1 map as we computed earlier? true
    - Does it have the same concentration as computed earlier? true



## Applying masks

Pharmacoknetic modelling of DCE-MRI requires concentration-time curves from:

- the tissue of interest (tumour in our cases) denoted as `ct`
- the feeding artery (arterial input function / AIF) denoted as `cp`
    + technically, it should be `cb` and we'll actually be using a vein instead of artery
- a healthy reference tissue (muscle) denoted as `crr`

Not all models require the same curves. For example, the well-established Tofts model only needs `ct` and `cp`, whereas the reference region model needs `ct` and `crr`.

Contours/masks for the tumour/muscle/blood-vessel were manually drawn and they can be downloaded by:

In [10]:
mask_folder = "./data/tcga-gbm-masks"
download_invivo_masks(destination = mask_folder)
# If they masks are already downloaded, then output will be `mask_folder`

"./data/tcga-gbm-masks"

In [11]:
# Get masks for the chosen study
mask = get_mask(study = chosen_study_uid, mask_folder = mask_folder)
keys(mask)

(:aif, :muscle, :tumour)

In [12]:
# Apply the masks to get ct, cp, and crr
# Note: computed.ct is concentration in entire volume, while ct is concentration in tumour.

hematocrit = 0.4 # For AIF (assumed value)

t = computed.t
ct = apply_mask(data = computed.ct, mask = mask.tumour)
cp = apply_mask(data = computed.ct, mask = mask.aif) ./ (1 - 0.4)
crr = apply_mask(data = computed.ct, mask = mask.muscle)

# cp and crr are averaged because only a single representative curve is needed from each.
cp = vec(mean(cp, dims=1))
crr = vec(mean(crr, dims=1));

## (Another) Wrapper function for all pre-processing steps

The pre-processing steps—i.e. T1 mapping, signal-concentration conversion, masking—are needed for each study.
Rather than repeating these steps every time, a wrapper function named `preprocess_dicom_to_mat` is used. 
This function applies all of the preceeding steps and saves the result as a MATLAB-compatible .mat file.

In [13]:
# `dicom_folders` was defined in the 2nd cell from the top
# `mask_folder` was defined in the "Applying masks" section
preprocessed_mat_files = preprocess_dicom_to_mat(destination = "./data/tcga-gbm-mat-test", 
    dicom_folders = dicom_folders, 
    mask_folder = mask_folder)
# dicom_folders only contains paths for a single study, so only a single .mat file will be produced.
# If the dicom files for all 8 studies were downloaded, then 8 .mat files would've been produced.

1-element Array{String,1}:
 "./data/tcga-gbm-mat-test/1.3.6.1.4.1.14519.5.2.1.4591.4001.304604545029494418165835320551.mat"

In [14]:
# The saved .mat file can be loaded into Julia by
mat_data = load_preprocessed_mat(preprocessed_mat_files[1])

println("""
Some information for the loaded mat_data
    - It is a named tuple with keys: $(keys(mat_data))
    - `masks` is a dictionary with keys: $(keys(mat_data.masks))
    - `relaxation` is a dictionary with keys: $(keys(mat_data.relaxation))
""")

Some information for the loaded mat_data
    - It is a named tuple with keys: (:t, :ct, :crr, :cp, :relaxation, :masks)
    - `masks` is a dictionary with keys: ["muscle", "tumour", "aif"]
    - `relaxation` is a dictionary with keys: ["M0", "T1"]



All 8 DCE-MRI studies can be pre-processed by running the following three lines:
```julia
dicom_folders = download_invivo_studies(destination = "./data/tcga-gbm-dicom")
mask_folder = download_invivo_masks(destination = "./data/tcga-gbm-masks")
preprocess_dicom_to_mat(destination = "./data/tcga-gbm-mat", dicom_folders = dicom_folders, mask_folder = mask_folder)
```

To save time, these steps were already done and the pre-processed files can be downloaded directly by:

In [15]:
mat_dir = "./data/tcga-gbm-mat"
download_invivo_preprocessed(destination = mat_dir)

"./data/tcga-gbm-mat"

The above command will not download any files if the `destination` folder already exists unless if `overwrite = true` is passed as an argument.

We can confirm that the .mat files are there by:

In [16]:
mat_files = readdir(mat_dir)

8-element Array{String,1}:
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.100057969162276274933613772317.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.269887096484012292940330991126.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.278082550121070125285213632206.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.304604545029494418165835320551.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.335353575986269052491315637674.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.365805576275232517344053939830.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.763554173270318063812534542847.mat"
 "1.3.6.1.4.1.14519.5.2.1.4591.4001.961791689281776173751323306588.mat"